In [0]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
import tarfile
import json
import numpy as np

Saving kaggle.json to kaggle.json


In [0]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [0]:
!kaggle datasets download -d yelp-dataset/yelp-dataset

100% 4.47G/4.48G [01:28<00:00, 10.0MB/s]
100% 4.48G/4.48G [01:28<00:00, 54.1MB/s]


In [0]:
!unzip "/content/yelp-dataset.zip"

Archive:  /content/yelp-dataset.zip
  inflating: Dataset_Agreement.pdf   
  inflating: yelp_academic_dataset_business.json  
  inflating: yelp_academic_dataset_checkin.json  
  inflating: yelp_academic_dataset_review.json  
  inflating: yelp_academic_dataset_tip.json  
  inflating: yelp_academic_dataset_user.json  


In [0]:
# Utilized some code from ([1] Zhongling Jiang, Build a Recommender System: Yelp Rating Prediction Example — Collaborative Filtering, (2019), Towards Data Science) just for loading initial json files in dataset

line_count = len(open("yelp_academic_dataset_review.json").readlines())
user_ids, business_ids, stars, dates, texts = [], [], [], [], []
with open("yelp_academic_dataset_review.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        user_ids += [blob["user_id"]]
        business_ids += [blob["business_id"]]
        stars += [blob["stars"]]
        dates += [blob["date"]]
        texts += [blob["text"]]
ratings_ = pd.DataFrame(
    {"user_id": user_ids, "business_id": business_ids, "rating": stars, "date": dates, "text": texts}
)

100%|██████████| 8021122/8021122 [02:54<00:00, 45933.15it/s]


In [0]:
user_counts = ratings_["user_id"].value_counts()
active_users = user_counts.loc[user_counts >= 5].index.tolist()
ratings_ = ratings_.loc[ratings_.user_id.isin(active_users)]

In [0]:
line_count = len(open("yelp_academic_dataset_user.json").readlines())
users, names, review_counts, since, friends, useful, \
            funny, cool, n_fans, years_elite, average_stars = [], [], [], [], [], [], [], [], [], [], []
with open("yelp_academic_dataset_user.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        users += [blob["user_id"]]
        names += [blob["name"]]
        review_counts += [blob["review_count"]]
        since += [blob["yelping_since"]]
        friends += [blob["friends"]]
        useful += [blob["useful"]]
        funny += [blob["funny"]]
        cool += [blob["cool"]]
        n_fans += [blob["fans"]]
        years_elite += [blob["elite"]]
        average_stars += [blob["average_stars"]]
        
users_ = pd.DataFrame(
    {"user_id": users, 
     "user_name": names,
     "user_review_count": review_counts,
     "user_yelp_since": since,
     "friends": friends,
     "useful_reviews": useful,
     "funny_reviews": funny,
     "cool_reviews": cool,
     "n_fans": n_fans,
     "years_elite": years_elite,
     "average_stars": average_stars
    }
)

100%|██████████| 1968703/1968703 [00:24<00:00, 79129.87it/s]


In [0]:

line_count = len(open("yelp_academic_dataset_business.json").readlines())
business_ids, names, addresses, cities, states, latitudes, longitudes, stars, \
        review_counts, is_open, categories = [], [], [], [], [], [], [], [], [], [], []
with open("yelp_academic_dataset_business.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        business_ids += [blob["business_id"]]
        names += [blob["name"]]
        addresses += [blob["address"]]
        cities += [blob["city"]]
        states += [blob["state"]]
        latitudes += [blob["latitude"]]
        longitudes += [blob["longitude"]]
        stars += [blob["stars"]]
        review_counts += [blob["review_count"]]
        is_open += [blob["is_open"]]
        categories += [blob["categories"]]
        
business_ = pd.DataFrame(
    {"business_id": business_ids, 
     "business_name": names,
     "business_address": addresses,
     "business_city": cities, 
     "business_state": states, 
     "business_latitude": latitudes, 
     "business_longitude": longitudes, 
     "stars": stars, 
     "review_counts": review_counts, 
     "is_open": is_open,
     "categories": categories}
)

100%|██████████| 209393/209393 [00:02<00:00, 78167.04it/s]


In [0]:
users_ = users_.loc[users_.user_id.isin(active_users)]

In [0]:
users_.head()

,user_id,user_name,user_review_count,user_yelp_since,friends,useful_reviews,funny_reviews,cool_reviews,n_fans,years_elite,average_stars
0,ntlvfPzc8eglqvk92iDIAw,Rafael,553,2007-07-06 03:27:11,"oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg...",628,225,227,14,,3.57
1,FOBRPlBHa3WPHFB5qYDlVg,Michelle,564,2008-04-28 01:29:25,"ly7EnE8leJmyqyePVYFlug, pRlR63iDytsnnniPb3AOug...",790,316,400,27,"2008,2009,2010,2011,2012,2013",3.84
3,QaELAmRcDc5TfJEylaaP8g,John,206,2008-09-20 00:08:14,"iog3Nyg1i4jeumiTVG_BSA, M92xWY2Vr9w0xoH8bPplfQ...",233,160,84,6,2009,3.08
5,z5_82komKV3mI4ASGe2-FQ,Steve,186,2007-02-27 07:09:40,"E-fXXmALnZTPmzzL6rV4cQ, L2g6vS1GDfqLEpiDOFfCkA...",642,192,155,15,2007,2.88
7,f4_MRNHvN-yRn7EA8YWRxg,Jennifer,822,2011-01-17 00:18:23,"c-Dja5bexzEWBufNsHfRrQ, 02HJNyOzzYXvEKVApJb8GQ...",4127,2446,2878,137,"2011,2012,2013,2014,2015,2016,2017,2018",3.63


In [0]:
business_['categories'] = business_['categories'].fillna('NULL')
restraunts = business_[business_.categories.str.contains('Restaurant')]

In [0]:
restraunts

,business_id,business_name,business_address,business_city,business_state,business_latitude,business_longitude,stars,review_counts,is_open,categories
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,40.110446,-88.233073,4.5,5,1,"Ethnic Food, Food Trucks, Specialty Food, Impo..."
20,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,4508 E Independence Blvd,Charlotte,NC,35.194894,-80.767442,3.0,5,0,"Food, Restaurants, Grocery, Middle Eastern"
24,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,44.010962,-79.448677,4.5,4,1,"Restaurants, Cheesesteaks, Poutineries"
25,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,43.820492,-79.398466,4.5,7,1,"Japanese, Fast Food, Food Court, Restaurants"
30,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,43.947011,-79.454862,3.0,3,1,"Persian/Iranian, Turkish, Middle Eastern, Rest..."
...,...,...,...,...,...,...,...,...,...,...,...
209381,xVpE01l6ZXdEtVf5PkRpDg,Julep,829 E Washington Ave,Madison,WI,43.081022,-89.374006,4.0,95,0,"Nightlife, Diners, Bars, Southern, Restaurants"
209383,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,27045 Lorain Rd,North Olmsted,OH,41.416568,-81.921396,3.0,13,0,"Restaurants, Sandwiches, Food, Breakfast & Bru..."
209385,hskVqZCPqy-omm9CHi44xQ,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,35.078538,-80.818358,2.0,16,1,"Restaurants, Pizza, Sandwiches, Fast Food, Chi..."
209388,9Q0fPWAjUweoFDk0kafuzQ,Nishi Sushi,9750 Weston Road,Vaughan,ON,43.838555,-79.559823,4.0,5,0,"Japanese, Sushi Bars, Restaurants"


In [0]:
restraunts.business_state.value_counts()

ON     16223
AZ     12131
NV      8346
QC      6228
OH      5914
NC      4656
PA      4260
AB      3245
WI      1794
IL       702
SC       427
NY        12
TX         4
CA         3
CO         2
BC         2
AL         1
HI         1
VT         1
OR         1
MB         1
XWY        1
FL         1
AR         1
VA         1
HPL        1
WA         1
NE         1
Name: business_state, dtype: int64

In [0]:
restraunts = restraunts[restraunts.business_state.str.contains('ON')]

In [0]:
restraunts.business_city.value_counts()

Toronto             8680
Mississauga         1612
Markham              925
Brampton             603
Richmond Hill        562
                    ... 
River Drive Park       1
Waterloo               1
Whiitby                1
North  York            1
Oakvile                1
Name: business_city, Length: 86, dtype: int64

In [0]:
#Toronto is the city with the most reviews 
restraunts = restraunts[restraunts.business_city=='Toronto']

In [0]:
restraunts.business_city.value_counts()

Toronto    8680
Name: business_city, dtype: int64

In [0]:
#delete irrelevant columns 
del restraunts['business_city']
del restraunts['business_state']
del restraunts['is_open']

In [0]:
restraunts.head()

,business_id,business_name,business_address,business_latitude,business_longitude,stars,review_counts,categories
38,rVBPQdeayMYht4Uv_FOLHg,Gourmet Burger Company,843 Kipling Avenue,43.633291,-79.531768,3.0,13,"Restaurants, Burgers, Food"
73,0QjROMVW9ACKjhSEfHqNCQ,Mi Mi Restaurant,688 Gerrard Street E,43.666376,-79.348773,4.0,116,"Vietnamese, Restaurants"
110,8k62wYhDVq1-652YbJi5eg,Tim Hortons,90 Adelaide Street W,43.649859,-79.382060,3.0,8,"Bagels, Donuts, Food, Cafes, Coffee & Tea, Res..."
149,0DnQh8SE8BSnvJltGCCiWg,Chick-N-Joy,3-1265 York Mills Road,43.765279,-79.326248,3.0,11,"Fast Food, Restaurants, Chicken Shop"
150,NLaK58WvlNQdUunSIkt-jA,Zav Coffee Shop & Gallery,2048 Danforth Avenue,43.685608,-79.313936,4.5,24,"Coffee & Tea, Restaurants, Sandwiches, Food"


In [0]:
scores = ratings_

In [0]:
del scores['text']

In [0]:
len(scores)

342768

In [0]:
restraunt_ids = restraunts.business_id.to_list()

In [0]:
len(restraunt_ids)

8680

In [0]:
scores = scores[scores.business_id.isin(restraunt_ids)]

In [0]:
len(scores)

342768

In [0]:
ratings_ = ratings_.loc[ratings_.business_id.isin(restraunt_ids)]

In [0]:
scores.head()

,user_id,business_id,rating,date
25,TZQSUDDcA4ek5gBd6BzcjA,qUWqjjjfpB2-4P3He5rsKw,4.0,2017-06-06 19:04:10
101,1kNsEAhGU8d8xugMuXJGFA,cDoo0Pf1d_P79Rq3ZNk-Mw,3.0,2013-07-13 00:57:45
114,Eg_VTAQwK3CxYTORNRWd2g,4m_hApwQ054v3ue_OxFmGw,5.0,2013-02-21 23:37:18
148,RRhco4irDXPk5EJmvuwr2w,PMPMHHLe9vFzcR67iCjgLA,5.0,2016-01-17 05:23:59
172,Kl6jNDAE7PG_VVQafIptxQ,IfUkm42SB9D0m_ZbkQ711A,4.0,2013-04-10 17:43:11


In [0]:
scores.user_id.value_counts()


CxDOIDnH8gp9KXzpBHJYXw    1130
Q9mA60HnY87C1TW5kjAZ6Q     624
O3pSxv1SyHpY4qi4Q16KzA     579
iRQ_YKpCBdaCwvc2X8_3NQ     483
ic-tyi1jElL_umxZVh8KNA     483
                          ... 
pqctgJVL6L00T-SaIWn2jQ       1
w3tXd4Y8eOnMUFterwzlhw       1
sdzXoKjU_-zeVrQX2Po5Jg       1
BW2-ad2rxu-vYBCn9fQm2Q       1
N2B-gmYnMtAdkQKMfCZcnw       1
Name: user_id, Length: 36249, dtype: int64

In [0]:
score_counts = scores["user_id"].value_counts()
active_scores = score_counts.loc[score_counts >= 5].index.tolist()
scores_meaningful = scores.loc[scores.user_id.isin(active_scores)]

In [0]:
users_meaningful = users_.loc[users_.user_id.isin(active_scores)]

In [0]:
len(users_meaningful)

17497

In [0]:
scores_meaningful.user_id.value_counts()

In [0]:
users_meaningful.to_csv('yelp_restraunt_raters.csv')
files.download('yelp_restraunt_raters.csv')

In [0]:
scores_meaningful.to_csv('yelp_restraunt_reviews.csv')
files.download('yelp_restraunt_reviews.csv')

In [0]:
restraunts.to_csv('yelp_restraunts.csv')
files.download('yelp_restraunts.csv')